In [1]:
import sqlite3
from sqlite3 import Error
# Declare variable conn
conn = None
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    global conn # Declare conn as global to be allow modification inside this function
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


C:\Users\wgerm\AppData\Local\Temp\ipykernel_24148\2343499927.py:15: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)


Question # 10   Produce a list of facilities with a total revenue less than 1000.                
The output of facility name and total revenue, sorted by revenue. Remember                
that there's a different cost for guests and members!

In [2]:
import pandas as pd

In [3]:
# Establish connection before the query
create_connection("sqlite_db_pythonsqlite.db")

2.6.0


C:\Users\wgerm\AppData\Local\Temp\ipykernel_24148\2343499927.py:15: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)


In [4]:
#Query 1  definition and execution
query1 = """
SELECT 
    f.name AS facility_name,
    SUM(
        CASE 
            WHEN b.memid = 0 THEN b.slots * f.guestcost 
            ELSE b.slots * f.membercost 
        END
    ) AS total_revenue
FROM 
    Bookings AS b
JOIN 
    Facilities AS f ON b.facid = f.facid
GROUP BY 
    f.name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue;
"""
# Execute the query and display results using pandas
if conn:  # Ensure the connection exists
    df = pd.read_sql_query(query1, conn)
    print(df)

else:
    print("Connection not established.")

   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


Question 11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [5]:
# Query 2 definition and execution 
query2 = """ 
SELECT 
    CONCAT(m1.surname,' ', m1.firstname) AS member_name,
    CASE 
        WHEN m2.surname != 'GUEST' THEN CONCAT(m2.surname, ' ', m2.firstname)
        ELSE 'No recommender'
    END AS recommender_name
FROM 
    Members AS m1
LEFT JOIN 
    Members AS m2
ON 
    m1.recommendedby = m2.memid
ORDER BY 
    m1.surname, m1.firstname;
    """
# Execute the query and display results using pandas
if conn:  # Ensure the connection exists
    df2 = pd.read_sql_query(query2, conn)
    print(df2)

    
else:
    print("Connection not established.")

                member_name   recommender_name
0            Bader Florence    Stibbons Ponder
1                Baker Anne    Stibbons Ponder
2             Baker Timothy     Farrell Jemima
3                Boothe Tim         Rownam Tim
4            Butters Gerald       Smith Darren
5               Coplin Joan      Baker Timothy
6             Crumpet Erica        Smith Tracy
7                Dare Nancy    Joplette Janice
8             Farrell David     No recommender
9            Farrell Jemima     No recommender
10              GUEST GUEST     No recommender
11          Genting Matthew     Butters Gerald
12                Hunt John  Purview Millicent
13              Jones David    Joplette Janice
14            Jones Douglas        Jones David
15          Joplette Janice       Smith Darren
16           Mackenzie Anna       Smith Darren
17             Owen Charles       Smith Darren
18             Pinker David     Farrell Jemima
19        Purview Millicent        Smith Tracy
20           

Question 12: Find the facilities with their usage by member, but not guests

In [6]:
# Query 3 : Definition and execution
query3 = """
SELECT 
    f.name AS facility_name,
    SUM(b.slots) AS total_usage_slots
FROM 
    Bookings AS b
INNER JOIN 
    Facilities AS f ON b.facid = f.facid
INNER JOIN 
    Members AS m ON b.memid = m.memid
WHERE 
    b.memid != 0  -- Exclude guest bookings
GROUP BY 
    f.name
ORDER BY 
    f.name;
    """
# Execute the query and display results using pandas
if conn:  # Ensure the connection exists
    df3 = pd.read_sql_query(query3, conn)
    print(df3)

else:
    print("Connection not established.")

     facility_name  total_usage_slots
0  Badminton Court               1086
1   Massage Room 1                884
2   Massage Room 2                 54
3       Pool Table                856
4    Snooker Table                860
5     Squash Court                418
6     Table Tennis                794
7   Tennis Court 1                957
8   Tennis Court 2                882


Question 13: Find the facilities' usage by month, but not guests

In [9]:
#Query 4 : definition and execution
query4 = """
SELECT 
    f.name AS facility_name,
    strftime('%Y-%m', b.starttime) AS month_year,
    SUM(b.slots) AS total_usage_slots
FROM 
    Bookings AS b
INNER JOIN 
    Facilities AS f ON b.facid = f.facid
WHERE 
    b.memid != 0  -- Exclude guest bookings
GROUP BY 
    f.name,
    month_year
ORDER BY 
    f.name,
    month_year;
    """
# Execute the query and display results using pandas
if conn:  # Ensure the connection exists
    df4 = pd.read_sql_query(query4, conn)
    print(df4)
    # Close the connection after execution
    conn.close()
else:
    print("Connection not established.")

      facility_name month_year  total_usage_slots
0   Badminton Court    2012-07                165
1   Badminton Court    2012-08                414
2   Badminton Court    2012-09                507
3    Massage Room 1    2012-07                166
4    Massage Room 1    2012-08                316
5    Massage Room 1    2012-09                402
6    Massage Room 2    2012-07                  8
7    Massage Room 2    2012-08                 18
8    Massage Room 2    2012-09                 28
9        Pool Table    2012-07                110
10       Pool Table    2012-08                303
11       Pool Table    2012-09                443
12    Snooker Table    2012-07                140
13    Snooker Table    2012-08                316
14    Snooker Table    2012-09                404
15     Squash Court    2012-07                 50
16     Squash Court    2012-08                184
17     Squash Court    2012-09                184
18     Table Tennis    2012-07                 98


In [ ]:
END.